In [1]:
#r "nuget:Microsoft.Data.Analysis, 0.21.1"
#r "nuget:Plotly.NET, 5.0.0"
#r "nuget:Plotly.NET.Interactive, 5.0.0"
#r "nuget:MathNet.Numerics, 5.0.0"
open MathNet.Numerics.Statistics
open Plotly.NET
open Microsoft.Data.Analysis

Installed Packages MathNet.Numerics, 5.0.0 Microsoft.Data.Analysis, 0.21.1 Plotly.NET, 5.0.0 Plotly.NET.Interactive, 5.0.0

Loading extensions from `C:\Users\elabo\.nuget\packages\plotly.net.interactive\5.0.0\lib\netstandard2.1\Plotly.NET.Interactive.dll`

Loading extensions from `C:\Users\elabo\.nuget\packages\microsoft.data.analysis\0.21.1\interactive-extensions\dotnet\Microsoft.Data.Analysis.Interactive.dll`

In [2]:
let df = DataFrame.LoadCsv("JoinedStats.csv")
df.Columns.Remove("Id")
df.Columns.Remove("club_id")

let marketValues = df["market_value_in_eur"] |> Seq.cast<single>
let positions = df["position"] |> Seq.cast<string>
let clubs = df["club_name"] |> Seq.cast<string>
let goals = df["goals"] |> Seq.cast<single>
let assists = df["assists"] |> Seq.cast<single>
let minutesPerGame = df["minutes_played"] |> Seq.cast<single>
let playerNames = df["name"] |> Seq.cast<string>
let redCards = df["red_cards"] |> Seq.cast<single>

In [3]:
Chart.BoxPlot(Y = marketValues, X = positions)
    |> Chart.withTitle "Player Market Value by Position"
    |> Chart.withSize(500, 350)

<!-- Plotly chart will be drawn inside this DIV -->

In [4]:
Chart.Violin(Y = marketValues, X = positions)
    |> Chart.withTitle "Player Market Value by Position"
    |> Chart.withSize(500, 350)

<!-- Plotly chart will be drawn inside this DIV -->

In [5]:
Chart.BoxPlot(Y = minutesPerGame, X = positions)
    |> Chart.withTitle "Playtime by Position"
    |> Chart.withSize(500, 350)

<!-- Plotly chart will be drawn inside this DIV -->

In [6]:
Chart.Point(x = minutesPerGame, 
            y = marketValues,
            MultiText = playerNames)
    |> Chart.withTitle "Market Valuation vs Playtime"
    |> Chart.withSize(600, 400)
    |> Chart.withXAxisStyle(TitleText = "Minutes per Game")
    |> Chart.withYAxisStyle(TitleText = "Market Valuation")

<!-- Plotly chart will be drawn inside this DIV -->

In [7]:
Chart.Point(x = goals,
            y = assists,
            MultiText = playerNames)
    |> Chart.withTitle "Goals vs Assists Per Game"
    |> Chart.withSize(600, 400)
    |> Chart.withXAxisStyle(TitleText = "Goals")
    |> Chart.withYAxisStyle(TitleText = "Assists")

<!-- Plotly chart will be drawn inside this DIV -->

In [8]:
Chart.Point(x = goals, y = assists, MultiText = playerNames)
    |> Chart.withTitle "Goals vs Assists Per Game"
    |> Chart.withSize(400, 400)
    |> Chart.withXAxisStyle(TitleText="Goals", MinMax=(0,2))
    |> Chart.withYAxisStyle(TitleText="Assists", MinMax=(0,2))

<!-- Plotly chart will be drawn inside this DIV -->

In [9]:
Chart.Point(x=goals, y=assists, MultiText=playerNames, MarkerColor= (Color.fromColorScaleValues minutesPerGame))
    |> Chart.withTitle "Goals vs Assists Colorized by Play Time"
    |> Chart.withSize(400, 400)
    |> Chart.withXAxisStyle(TitleText="Goals", MinMax=(-0.1, 1.6))
    |> Chart.withYAxisStyle(TitleText="Assists", MinMax=(-0.1, 1.6))
    |> Chart.withMarkerStyle(ShowScale=true)

<!-- Plotly chart will be drawn inside this DIV -->

In [10]:
Chart.Point3D(x=goals, y=assists, z=marketValues, MultiText=playerNames, MarkerColor=(Color.fromColorScaleValues marketValues))
    |> Chart.withTitle "Goals vs Assists vs Valuation"
    |> Chart.withXAxisStyle(TitleText="Goals", Id=StyleParam.SubPlotId.Scene 1)
    |> Chart.withYAxisStyle(TitleText="Assists", Id=StyleParam.SubPlotId.Scene 1)
    |> Chart.withZAxisStyle(TitleText="MarketValue")
    |> Chart.withSize(800, 600) 

<!-- Plotly chart will be drawn inside this DIV -->

In [11]:
Correlation.Pearson(goals |> Seq.map float, assists |> Seq.map float)

0.4374489344525743

In [12]:
Correlation.Pearson(assists |> Seq.map float, redCards |> Seq.map float)

-0.04253652627440338

In [13]:
let columnsToAnalyze = df.Columns 
                    |> Seq.filter (fun c -> c.GetType() = typeof<SingleDataFrameColumn>)
                    |> Seq.map (fun col -> col.Name)
                    |> Array.ofSeq
let numColumns = Array.length columnsToAnalyze


In [30]:
let results : float[][] = Array.zeroCreate numColumns
for y = 0 to numColumns - 1 do
    let colY = df[columnsToAnalyze[y]] 
            |> Seq.cast<single>
            |> Seq.map float
    let rowResults : float[] = Array.create numColumns nan
    results.[y] <- rowResults

    for x = 0 to y do 
    let colX = df[columnsToAnalyze[x]]
            |> Seq.cast<single>
            |> Seq.map float
    rowResults. [x] <- Correlation.Pearson(colX, colY)

In [31]:
Chart.Heatmap(zData = results, ReverseYAxis = true, X = columnsToAnalyze, Y = columnsToAnalyze)
    |> Chart.withTitle( "Correlation Matrix")

<!-- Plotly chart will be drawn inside this DIV -->

Error: input.fsx (1,14)-(1,15) typecheck error The value or constructor 'y' is not defined.
input.fsx (5,5)-(5,15) typecheck error The value or constructor 'rowResults' is not defined. Maybe you want one of the following:
   results
input.fsx (5,5)-(5,55) typecheck error The operator 'expr.[idx]' has been used on an object of indeterminate type based on information prior to this program point. Consider adding further type constraints